Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

output_csv_path = "/content/drive/MyDrive/detectron2_metrics.csv"

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
import os, shutil, json
from pathlib import Path
import numpy as np

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nadyana120876","key":"f8464b9a119178d1ac11edd17e44167a"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nadyana120876/dataset-with-coco
!unzip dataset-with-coco.zip -d /content/import-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/import-dataset/dataset/images/valid/661621d4a14e487c8c74354134a29710.jpg  
  inflating: /content/import-dataset/dataset/images/valid/661bef3ef00b4bb7b29e797ea4db91c2.jpg  
  inflating: /content/import-dataset/dataset/images/valid/662ab83988d14ebfaf059b2980cad943.jpg  
  inflating: /content/import-dataset/dataset/images/valid/662e821b90574df08e8f21ecaea3a6c7.jpg  
  inflating: /content/import-dataset/dataset/images/valid/6638ad392fb843119c3a2b626efb6c1f.jpg  
  inflating: /content/import-dataset/dataset/images/valid/6638ede8fcba41deaf2f44c77da8cc82.jpg  
  inflating: /content/import-dataset/dataset/images/valid/664367a2a1a04e31acc9df46d10cbb65.jpg  
  inflating: /content/import-dataset/dataset/images/valid/664a10815646427cb123b7f194d01714.jpg  
  inflating: /content/import-dataset/dataset/images/valid/666107a01af8430c8c347e7699908912.jpg  
  inflating: /content/import-dataset/dataset/images/valid/666f837887f740a982

move dataset

In [ ]:
dataset_root = "/content/dataset"
os.makedirs(f"{dataset_root}/images", exist_ok=True)
os.makedirs(f"{dataset_root}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_root}/labels/valid", exist_ok=True)

og_train_dir = "/content/import-dataset/dataset/images/train"
shutil.move(og_train_dir, f"{dataset_root}/images")

og_val_dir = "/content/import-dataset/dataset/images/valid"
shutil.move(og_val_dir, f"{dataset_root}/images")

og_test_dir = "/content/import-dataset/dataset/images/test"
shutil.move(og_test_dir, f"{dataset_root}/images")

'/content/dataset/images/test'

In [ ]:
os.makedirs(f"{dataset_root}/annotations", exist_ok=True)
shutil.move("/content/import-dataset/json/train.json", f"{dataset_root}/annotations/train.json")
shutil.move("/content/import-dataset/json/val.json", f"{dataset_root}/annotations/valid.json")
shutil.move("/content/import-dataset/json/test.json", f"{dataset_root}/annotations/test.json")

'/content/dataset/annotations/test.json'

fix dataset

In [ ]:
import json
import os
import tqdm

def remap_coco_json(input_path, output_path):
    with open(input_path, "r") as f:
        data = json.load(f)
    if "info" not in data:
        data["info"] = {
            "description": "Custom Dataset",
            "version": "1.0",
            "year": 2024
        }
    if "licenses" not in data:
        data["licenses"] = []

    if "annotations" not in data or "categories" not in data:
        raise ValueError(f"Missing 'annotations' or 'categories' in {input_path}")
    used_cat_ids = sorted({ann["category_id"] for ann in data["annotations"]})
    print(f"Number of used categories: {len(used_cat_ids)}")

    id_map = {old_id: new_id for new_id, old_id in enumerate(used_cat_ids)}

    for ann in data["annotations"]:
        ann["category_id"] = id_map[ann["category_id"]]

    new_categories = []
    for cat in data["categories"]:
        if cat["id"] in id_map:
            cat["id"] = id_map[cat["id"]]
            new_categories.append(cat)
    data["categories"] = new_categories

    with open(output_path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"Saved remapped file to {output_path}")
    print("NUM_CLASSES should be:", len(used_cat_ids))


remap_coco_json("/content/dataset/annotations/train.json",
                "/content/dataset/annotations/train_remapped.json")

remap_coco_json("/content/dataset/annotations/valid.json",
                "/content/dataset/annotations/valid_remapped.json")

remap_coco_json("/content/dataset/annotations/test.json",
                "/content/dataset/annotations/test_remapped.json")

Number of used categories: 555
Saved remapped file to /content/dataset/annotations/train_remapped.json
NUM_CLASSES should be: 555
Number of used categories: 555
Saved remapped file to /content/dataset/annotations/valid_remapped.json
NUM_CLASSES should be: 555
Number of used categories: 555
Saved remapped file to /content/dataset/annotations/test_remapped.json
NUM_CLASSES should be: 555


Install

In [ ]:
!pip install -U torch torchvision
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

register_coco_instances(
    "my_dataset_train",
    {},
    "/content/dataset/annotations/train_remapped.json",
    "/content/dataset/images/train"
)
register_coco_instances(
    "my_dataset_val",
    {},
    "/content/dataset/annotations/valid_remapped.json",
    "/content/dataset/images/valid"
)
register_coco_instances(
    "my_dataset_test",
    {},
    "/content/dataset/annotations/test_remapped.json",
    "/content/dataset/images/test"
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# check 
import os
import detectron2
import inspect

d2_path = os.path.dirname(inspect.getfile(detectron2))
print(d2_path)

print(os.listdir(os.path.join(d2_path, "projects")))


/usr/local/lib/python3.12/dist-packages/detectron2
['point_rend', 'panoptic_deeplab', '__init__.py', '__pycache__', 'deeplab']


In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/vitdet_outputs/model_0096719.pth"


cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 555
cfg.MODEL.MASK_ON = False


cfg.OUTPUT_DIR = "/content/drive/MyDrive/vitdet_outputs"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


# increase
cfg.SOLVER.IMS_PER_BATCH = 32
cfg.DATALOADER.NUM_WORKERS = 6
cfg.SOLVER.BASE_LR = 0.00025

NameError: name 'os' is not defined

In [ ]:
import os
train_dir = "/content/dataset/images/train"
num_images = sum(len(files) for _, _, files in os.walk(train_dir))
batch_size = cfg.SOLVER.IMS_PER_BATCH

# iterations_per_epoch = num_images // batch_size
iterations_per_epoch = num_images // 24

cfg.SOLVER.MAX_ITER = iterations_per_epoch * 70
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0001
cfg.SOLVER.LOG_PERIOD = iterations_per_epoch
cfg.TEST.EVAL_PERIOD = iterations_per_epoch

# disable eval 
cfg.TEST.EVAL_PERIOD = iterations_per_epoch * 0   
cfg.SOLVER.CHECKPOINT_PERIOD = iterations_per_epoch * 5

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.engine import hooks
from detectron2.config import LazyCall as L
from detectron2.engine import DefaultTrainer
import csv
import os


class TrainerWithBestModel(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "eval")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    def __init__(self, cfg):
        super().__init__(cfg)
        self.best_ap = -1  

    def after_step(self):
        super().after_step()

        if (self.iter + 1) % cfg.TEST.EVAL_PERIOD == 0:
            results = self.test(self.cfg, self.model)  

            self.save_results_to_csv(results)

            current_ap = results.get("bbox", {}).get("AP", None)
            if current_ap is not None and current_ap > self.best_ap:
                self.best_ap = current_ap
                self.checkpointer.save("model_best")
                print(f"✅ Saved new best model at iter {self.iter+1}, AP={self.best_ap:.4f}")

    def save_results_to_csv(self, results, csv_path="/content/drive/MyDrive/detectron2_metrics.csv"):
        file_exists = os.path.isfile(csv_path)

        flattened = {"iteration": self.iter + 1}
        for key, value in results.items():
            if isinstance(value, dict):
                for subkey, subval in value.items():
                    flattened[f"{key}_{subkey}"] = subval
            else:
                flattened[key] = value

        with open(csv_path, mode='a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=flattened.keys())
            if not file_exists:
                writer.writeheader()
            writer.writerow(flattened)

In [ ]:
print("Backbone:", cfg.MODEL.BACKBONE.NAME)
print("ResNet Depth:", cfg.MODEL.RESNETS.DEPTH)

In [ ]:
from detectron2.engine import DefaultTrainer

trainer = DefaultTrainer(cfg)
# trainer = TrainerWithBestModel(cfg)

trainer.resume_or_load(resume=True)
trainer.train()


[11/01 16:37:59 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.12/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W1101 16:38:12.017000 4996 torch/fx/_symbolic_trace.py:52] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.


[11/01 16:38:32 d2.utils.events]:  eta: 9:26:01  iter: 96739  total_loss: 0.1287  loss_cls: 0.07841  loss_box_reg: 0.0444  loss_rpn_cls: 0.0003704  loss_rpn_loc: 0.003012    time: 1.0518  last_time: 1.0419  data_time: 0.1615  last_data_time: 0.1308   lr: 0.0002  max_mem: 18606M
[11/01 16:39:00 d2.utils.events]:  eta: 9:25:40  iter: 96759  total_loss: 0.1219  loss_cls: 0.07358  loss_box_reg: 0.04376  loss_rpn_cls: 0.0005135  loss_rpn_loc: 0.002804    time: 1.0504  last_time: 1.0293  data_time: 0.1417  last_data_time: 0.1247   lr: 0.0002  max_mem: 18606M
[11/01 16:39:21 d2.utils.events]:  eta: 9:20:53  iter: 96779  total_loss: 0.1215  loss_cls: 0.07488  loss_box_reg: 0.04212  loss_rpn_cls: 0.0003828  loss_rpn_loc: 0.002865    time: 1.0496  last_time: 1.0406  data_time: 0.1431  last_data_time: 0.1338   lr: 0.0002  max_mem: 18606M
[11/01 16:39:41 d2.utils.events]:  eta: 9:21:13  iter: 96799  total_loss: 0.1202  loss_cls: 0.07515  loss_box_reg: 0.04195  loss_rpn_cls: 0.0003252  loss_rpn_loc

In [ ]:
# import os
# import csv
# from detectron2.engine import DefaultTrainer
# from detectron2.config import get_cfg
# from detectron2.data import build_detection_test_loader
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset

# dataset_name = cfg.DATASETS.TEST[0]
# evaluator = COCOEvaluator(dataset_name, cfg, False, output_dir=os.path.join(cfg.OUTPUT_DIR, "eval"))
# val_loader = build_detection_test_loader(cfg, dataset_name)

# ## evaluation
# trainer = TrainerWithBestModel(cfg)
# trainer.resume_or_load(resume=False)
# results = inference_on_dataset(trainer.model, val_loader, evaluator)

train using lazy config??

In [ ]:
# !python tools/lazyconfig_train_net.py --config-file my_vitdet_cfg.py

In [ ]:
# from detectron2.config import instantiate
# from detectron2.data import build_detection_train_loader

# try:
#     # Use the trainer from the config, but replace the class
#     cfg.train.trainer.target = TrainerWithBestModel
#     trainer = instantiate(cfg.train)
# except:
#   print("Default LazyConfig instantiation failed. Falling back to manual build.")
#   trainer = TrainerWithBestModel(cfg)
#   trainer.resume_or_load(resume=False)
#   trainer.train()

eval

In [ ]:
# from detectron2.engine import DefaultTrainer
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# from detectron2.config import get_cfg
# from detectron2 import model_zoo

# trainer = TrainerWithVal(cfg)
# trainer.resume_or_load(resume=False)

# evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="./output_vitdet/eval")
# val_loader = build_detection_test_loader(cfg, "my_dataset_val")
# inference_on_dataset(trainer.model, val_loader, evaluator)